In [1]:
import pandas as pd
import numpy as np
main = '/Users/aklat/Downloads/Thsesis/Datasets/Moral Machine Data/SharedResponses.csv'


In [2]:
import pandas as pd

# load data with certain columns
main = '/Users/aklat/Downloads/Thsesis/Datasets/Moral Machine Data/SharedResponses.csv'
all_columns = pd.read_csv(main, nrows=0).columns.tolist()
selected_columns = all_columns[:8] + all_columns[14:17] + all_columns[-20:]

# create a new dataframe
df_main = pd.DataFrame()

# load data
for chunk in pd.read_csv(main, usecols=selected_columns, chunksize=1000000, low_memory=False):
    df_main = pd.concat([df_main, chunk], ignore_index=True)
print(df_main.head())

          ResponseID              ExtendedSessionID        UserID  \
0  2222bRQqBTZ6dLnPH    32757157_6999801415950060.0  6.999801e+15   
1  2222sJk4DcoqXXi98        1043988516_3525281295.0  3.525281e+09   
2  2223CNmvTr2Coj4wp  -1613944085_422160228641876.0  4.221602e+14   
3  2223Xu54ufgjcyMR3   1425316635_327833569077076.0  3.278336e+14   
4  2223jMWDEGNeszivb  -1683127088_785070916172117.0  7.850709e+14   

   ScenarioOrder  Intervention  PedPed  Barrier  CrossingSignal  \
0              7             0       0        0               1   
1              2             0       0        0               0   
2             10             0       1        0               1   
3             11             0       0        1               0   
4              8             0       1        0               2   

   NumberOfCharacters  DiffNumberOFCharacters  ...  LargeMan Criminal  \
0                 5.0                     0.0  ...       0.0      0.0   
1                 1.0               

In [3]:
df_main.shape

(70332355, 31)

In [4]:
df_main.to_csv('main_feature_1.csv')

In [5]:
import pandas as pd
f1_path = '/Users/aklat/Downloads/Thsesis/Datasets/Moral Machine Data/main_feature_1.csv'
f1=pd.read_csv(f1_path,nrows=190)
f1.columns


Index(['Unnamed: 0', 'ResponseID', 'ExtendedSessionID', 'UserID',
       'ScenarioOrder', 'Intervention', 'PedPed', 'Barrier', 'CrossingSignal',
       'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'Man', 'Woman',
       'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless',
       'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive',
       'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor',
       'MaleDoctor', 'Dog', 'Cat'],
      dtype='object')

In [10]:
import pandas as pd

file_path = '/Users/aklat/Downloads/Thsesis/Datasets/Moral Machine Data/SharedResponsesSurvey.csv'

# extract 'UserID'
df = pd.read_csv(file_path, usecols=["UserID"])

# remove userid's duplicates
df_unique = df.drop_duplicates()
print(df_unique.shape)

(463675, 1)


In [14]:
import pandas as pd

f1_path = '/Users/aklat/Downloads/Thsesis/Datasets/Moral Machine Data/main_feature_1.csv'

merged_chunks = []

chunksize = 100000

# merge 32 columns' main data with user data through 'UserID'
for chunk in pd.read_csv(f1_path, chunksize=chunksize):
    merged_chunk = pd.merge(df_unique, chunk, on='UserID', how='inner')
    merged_chunks.append(merged_chunk)

final_merged_df = pd.concat(merged_chunks, ignore_index=True)

print(final_merged_df.shape)


/var/folders/x5/b6qvk1zx3dg9d_8_8zt4hf3m0000gn/T/ipykernel_23210/3050306158.py:13: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(f1_path, chunksize=chunksize):
/var/folders/x5/b6qvk1zx3dg9d_8_8zt4hf3m0000gn/T/ipykernel_23210/3050306158.py:13: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(f1_path, chunksize=chunksize):
/var/folders/x5/b6qvk1zx3dg9d_8_8zt4hf3m0000gn/T/ipykernel_23210/3050306158.py:13: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(f1_path, chunksize=chunksize):
/var/folders/x5/b6qvk1zx3dg9d_8_8zt4hf3m0000gn/T/ipykernel_23210/3050306158.py:13: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(f1_path, chunksize=chunksize):
/var/folders/x5/b6qvk1zx3dg9d_8_

(13127377, 32)


In [16]:
final_merged_df.to_csv('main_link_user.csv')

In [44]:
import pandas as pd

# check structure of current data, filter those whose number of rows != 26 & number of unique ResponseID !=13
df=final_merged_df

grouped_df = df.groupby('UserID').agg(total_rows=('UserID', 'size'), unique_responseids=('ResponseID', 'nunique'))
result_df = grouped_df[(grouped_df['total_rows'] != 26) & (grouped_df['unique_responseids'] != 13)]

print(result_df)

# 340k out of 460k users' data is not good

              total_rows  unique_responseids
UserID                                      
2.173558e+09          22                  11
2.271406e+09         202                 104
2.371697e+09         129                  68
2.372729e+09         575                 299
2.423489e+09          24                  12
...                  ...                 ...
9.999948e+15          22                  11
9.999965e+15          24                  12
9.999990e+15          22                  12
9.999993e+15          99                  52
9.999993e+15          23                  12

[339406 rows x 2 columns]


In [45]:
import pandas as pd
import numpy as np

# analyze distribution of users of which number of data is abnormal
bins = np.arange(0, result_df['total_rows'].max() + 26, 26)
result_df['total_rows_bin'] = pd.cut(result_df['total_rows'], bins)
grouped = result_df.groupby('total_rows_bin').size()
grouped = grouped[grouped != 0]
print(grouped)

total_rows_bin
(0, 26]           238509
(26, 52]           75089
(52, 78]           16222
(78, 104]           5108
(104, 130]          2035
                   ...  
(25896, 25922]         1
(50336, 50362]         1
(52728, 52754]         1
(53872, 53898]         1
(61802, 61828]         1
Length: 61, dtype: int64


/var/folders/x5/b6qvk1zx3dg9d_8_8zt4hf3m0000gn/T/ipykernel_23210/79593751.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['total_rows_bin'] = pd.cut(result_df['total_rows'], bins)
/var/folders/x5/b6qvk1zx3dg9d_8_8zt4hf3m0000gn/T/ipykernel_23210/79593751.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = result_df.groupby('total_rows_bin').size()


Strategy of filtering data:

Step 1: Group by UserID and calculate the total data volume and unique ResponseID number for each user. If total=26 and unique=13, keep all data of these users.
Step 2: If total<26, filter out all rows where ResponseID appears twice and the Saved column is 0 and 1 respectively, and keep this part of the data.
Step 3: If total>26, randomly select 13 ResponseIDs, and these ResponseIDs appear twice and the Saved column is 0 and 1 respectively.

In [125]:
import pandas as pd
import numpy as np

# group by UserID and calculate total rows per user
grouped_df = final_merged_df.groupby('UserID').agg(
    total_rows=('UserID', 'size'),  
    unique_responseids=('ResponseID', 'nunique')  
).reset_index()

# Step 1: keep data that total = 26 & unique = 13 
condition_1 = grouped_df[(grouped_df['total_rows'] == 26) & (grouped_df['unique_responseids'] == 13)]
filtered_df_1 = final_merged_df[final_merged_df['UserID'].isin(condition_1['UserID'])]

# Step 2: for users whose total < 26，make sure responseid is paired
condition_2 = grouped_df[grouped_df['total_rows'] < 26]

# total < 26
filtered_df_2 = final_merged_df[final_merged_df['UserID'].isin(condition_2['UserID'])]

# filter ResponseID = 2 & Saved contains 0 and 1
def filter_responseid_group(group):
    valid_responses = group.groupby('ResponseID').filter(lambda x: (len(x) == 2) and (x['Saved'].nunique() == 2))
    return valid_responses

filtered_df_2 = filtered_df_2.groupby('UserID').apply(filter_responseid_group).reset_index(drop=True)

# Step 3: for users whose total > 26 的用户，pick 13 valid ResponseID randomaly
condition_3 = grouped_df[grouped_df['total_rows'] > 26]

# total > 26
filtered_df_3 = final_merged_df[final_merged_df['UserID'].isin(condition_3['UserID'])]

def random_sample(group):
    valid_responses = group.groupby('ResponseID').filter(lambda x: (len(x) == 2) and (x['Saved'].nunique() == 2))
    
    if len(valid_responses['ResponseID'].unique()) > 13:
        selected_responseids = np.random.choice(valid_responses['ResponseID'].unique(), size=13, replace=False)
        
        sample_responses = valid_responses[valid_responses['ResponseID'].isin(selected_responseids)]
        return sample_responses
    else:
        return valid_responses

filtered_df_3 = filtered_df_3.groupby('UserID').apply(random_sample).reset_index(drop=True)

# concat
final_filtered_df = pd.concat([filtered_df_1, filtered_df_2, filtered_df_3], ignore_index=True)

print(final_filtered_df)

               UserID  Unnamed: 0         ResponseID  \
0        7.981091e+15       94911  2A8SAxAa6dLGK4bGm   
1        9.964404e+15       51566  26Q8hd6RncCNv6gut   
2        9.964404e+15       99848  2AZSA9oesRcN5utcx   
3        5.209783e+15       35711  254hWwD8y4bcCtxp8   
4        7.888200e+14       43929  25mYXXQZzcAg2sG5E   
...               ...         ...                ...   
9704245  9.999993e+15    59198029  vv48Mu99wjf7tbmES   
9704246  9.999993e+15    61751683  zuC33ubGwCsfsCyCq   
9704247  9.999993e+15    62426718  ndyokXboZiRhJiSnZ   
9704248  9.999993e+15    67721180  vv48Mu99wjf7tbmES   
9704249  9.999993e+15    70264960  zuC33ubGwCsfsCyCq   

                      ExtendedSessionID  ScenarioOrder  Intervention  PedPed  \
0        -2119938054_7981090920342526.0             11             0       1   
1        -2119627065_9964404106140136.0              6             0       1   
2        -2119627065_9964404106140136.0             10             0       0   
3      

In [126]:
final_filtered_df.to_csv('clean_predata_1020.csv')

In [127]:
import pandas as pd
import numpy as np

# transform float in column[11:] to int
cols_to_convert = final_filtered_df.columns[11:]  

for col in cols_to_convert:
    final_filtered_df[col] = final_filtered_df[col].astype(int)

In [128]:
cols_to_convert = final_filtered_df.columns[9:10]
final_filtered_df[col] = final_filtered_df[col].astype(int)

In [129]:
df = final_filtered_df
cols = df.columns[11:]

# calculate unique values in each column
for col in cols:
    value_counts = df[col].value_counts()
    print(f"column; {col}")
    print(value_counts)
    print("-" * 20)

column; Saved
Saved
0    4852125
1    4852125
Name: count, dtype: int64
--------------------
column; Man
Man
0    7299193
1    1918297
2     410689
3      68358
4       7351
5        362
Name: count, dtype: int64
--------------------
column; Woman
Woman
0    7296524
1    1921144
2     410432
3      68411
4       7375
5        364
Name: count, dtype: int64
--------------------
column; Pregnant
Pregnant
0    9141417
1     519705
2      41258
3       1825
4         43
5          2
Name: count, dtype: int64
--------------------
column; Stroller
Stroller
0    9148101
1     513117
2      41180
3       1802
4         49
5          1
Name: count, dtype: int64
--------------------
column; OldMan
OldMan
0    8249280
1    1090017
2     274991
3      74345
4      14342
5       1275
Name: count, dtype: int64
--------------------
column; OldWoman
OldWoman
0    8249616
1    1090784
2     273766
3      74475
4      14382
5       1227
Name: count, dtype: int64
--------------------
column; Boy
Boy
0    

In [130]:
final_filtered_df.to_csv('clean_predata_1020.csv')

In [131]:
final_filtered_df.columns

Index(['UserID', 'Unnamed: 0', 'ResponseID', 'ExtendedSessionID',
       'ScenarioOrder', 'Intervention', 'PedPed', 'Barrier', 'CrossingSignal',
       'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'Man', 'Woman',
       'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless',
       'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive',
       'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor',
       'MaleDoctor', 'Dog', 'Cat'],
      dtype='object')

Vectorization
gender_vec = {'unknown':0,'male':1,'female':2}
age_vec = {'unknown':0,'baby':1,'young':2,'adult':3,'old':4}
bodt_type_vec={'normal':0,'large':1}
profession_vec = {'crinimal':-2,'homeless':-1,'unknown':0,'AthDocExe':1}
pet_vec = {'human':0,'animal':1}
pregnant = {'normal':0,'pregnant':1}

In [134]:
import numpy as np
import pandas as pd

# vectorize
def vectorize_character(character_type, count):
    if count == 0:
        return None  
    
    if character_type == 'Man':
        gender_vec = [1]
        age_vec = [3]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'Woman':
        gender_vec = [2]
        age_vec = [3]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'Pregnant':
        gender_vec = [2]
        age_vec = [3]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [1]
    elif character_type == 'Stroller':
        gender_vec = [0]
        age_vec = [1]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'Boy':
        gender_vec = [1]
        age_vec = [2]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'Girl':
        gender_vec = [2]
        age_vec = [2]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'OldMan':
        gender_vec = [1]
        age_vec = [4]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'OldWoman':
        gender_vec = [2]
        age_vec = [4]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'Homeless':
        gender_vec = [0]
        age_vec = [0]
        body_type_vec = [0]
        profession_vec = [-1]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'Criminal':
        gender_vec = [0]
        age_vec = [0]
        body_type_vec = [0]
        profession_vec = [-2]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'LargeWoman':
        gender_vec = [2]
        age_vec = [3]
        body_type_vec = [1]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'LargeMan':
        gender_vec = [1]
        age_vec = [3]
        body_type_vec = [1]
        profession_vec = [0]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'MaleExecutive' or character_type == 'MaleAthlete' or character_type == 'MaleDoctor':
        gender_vec = [1]
        age_vec = [3]
        body_type_vec = [0]
        profession_vec = [1]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'FemaleDoctor' or character_type == 'FemaleExecutive' or character_type == 'FemaleAthlete':
        gender_vec = [2]
        age_vec = [3]
        body_type_vec = [0]
        profession_vec = [1]
        pet_vec = [0]
        pregnant_vec = [0]
    elif character_type == 'Dog':
        gender_vec = [0]
        age_vec = [0]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [1]
        pregnant_vec = [0]
    elif character_type == 'Cat':
        gender_vec = [0]
        age_vec = [0]
        body_type_vec = [0]
        profession_vec = [0]
        pet_vec = [1]
        pregnant_vec = [0]
    
    return {
        "gender_vec": np.tile(gender_vec, (count, 1)).sum(axis=0),
        "age_vec": np.tile(age_vec, (count, 1)).sum(axis=0),
        "body_type_vec": np.tile(body_type_vec, (count, 1)).sum(axis=0),
        "profession_vec": np.tile(profession_vec, (count, 1)).sum(axis=0),
        "pet_vec": np.tile(pet_vec, (count, 1)).sum(axis=0),
        "pregnant_vec": np.tile(pregnant_vec, (count, 1)).sum(axis=0)
    }

# add ntervention、Barrier、CrossingSignal
def vectorize_scene_features(row, number_of_characters):
    cross_signal = row['CrossingSignal']
    if cross_signal == 2:
        cross_signal = -1
    
    scene_vector = np.tile([row['Intervention'], row['Barrier'], cross_signal], number_of_characters)
    return {
        "Intervention_vec": scene_vector[0],
        "Barrier_vec": scene_vector[1],
        "CrossingSignal_vec": scene_vector[2]
    }

# process each row
def vectorize_row(row):
    total_vector = {
        "gender_vec": 0,
        "age_vec": 0,
        "body_type_vec": 0,
        "profession_vec": 0,
        "pet_vec": 0,
        "pregnant_vec": 0
    }
    
    for character_type in ['Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless',
                           'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete',
                           'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']:
        count = row[character_type]
        char_vecs = vectorize_character(character_type, count)
        if char_vecs:
            for key in total_vector:
                total_vector[key] += char_vecs[key]
    
    scene_vecs = vectorize_scene_features(row, int(row['NumberOfCharacters']))
    
    return {**total_vector, **scene_vecs}

# keep part of orignial columns
def create_new_dataframe(df):
    new_rows = []
    
    for _, row in df.iterrows():
        vectorized_data = vectorize_row(row)
        new_row = {
            "UserID": row["UserID"],
            "ResponseID": row["ResponseID"],
            "ExtendedSessionID": row["ExtendedSessionID"],
            "ScenarioOrder": row["ScenarioOrder"],
            "NumberOfCharacters": row["NumberOfCharacters"],
            "DiffNumberOFCharacters": row["DiffNumberOFCharacters"],
            "Saved": row["Saved"]
        }
        new_row.update(vectorized_data)
        new_rows.append(new_row)
    
    return pd.DataFrame(new_rows)


df = final_filtered_df
vectorized_df = create_new_dataframe(df)

print(vectorized_df.head())


         UserID         ResponseID               ExtendedSessionID  \
0  7.981091e+15  2A8SAxAa6dLGK4bGm  -2119938054_7981090920342526.0   
1  9.964404e+15  26Q8hd6RncCNv6gut  -2119627065_9964404106140136.0   
2  9.964404e+15  2AZSA9oesRcN5utcx  -2119627065_9964404106140136.0   
3  5.209783e+15  254hWwD8y4bcCtxp8  -2117293878_5209782729596508.0   
4  7.888200e+14  25mYXXQZzcAg2sG5E   -2112010153_788820015732199.0   

   ScenarioOrder  NumberOfCharacters  DiffNumberOFCharacters  Saved  \
0             11                 3.0                     0.0      0   
1              6                 4.0                     1.0      0   
2             10                 3.0                     2.0      0   
3              6                 1.0                     0.0      1   
4              6                 1.0                     2.0      1   

  gender_vec age_vec body_type_vec profession_vec pet_vec pregnant_vec  \
0        [5]     [8]           [0]            [0]     [0]          [0]   
1   

In [135]:
vectorized_df.shape

(9704250, 16)

In [137]:
# multiply number of characters to signal/intervention/barrier
df2 = vectorized_df
df2['CrossingSignal_vec']=df2['CrossingSignal_vec']*df2['NumberOfCharacters']
df2['Intervention_vec']=df2['Intervention_vec']*df2['NumberOfCharacters']
df2['Barrier_vec']=df2['Barrier_vec']*df2['NumberOfCharacters']

In [138]:
df2.columns

Index(['UserID', 'ResponseID', 'ExtendedSessionID', 'ScenarioOrder',
       'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'gender_vec',
       'age_vec', 'body_type_vec', 'profession_vec', 'pet_vec', 'pregnant_vec',
       'Intervention_vec', 'Barrier_vec', 'CrossingSignal_vec'],
      dtype='object')

In [139]:
# in vecctorized_df, these columns' data are like; [number], so remove []
cols_to_convert = ['gender_vec', 'age_vec', 'body_type_vec', 'profession_vec', 'pet_vec', 'pregnant_vec']
for col in cols_to_convert:
    df2[col] = df2[col].apply(lambda x: int(x[0]))

In [140]:
df2.to_csv ('vectorized data_1.csv')

In [141]:
# # calculate each responseid's number of rows
# responseid_count = df2.groupby('ResponseID').size()

# # print distribution
# distribution = responseid_count.value_counts().sort_index()

# print(distribution)


2    4852125
Name: count, dtype: int64


Prediction through RankNet

In [142]:
import pandas as pd

df = vectorized_df

grouped = df.groupby('ResponseID')

# create RankNet's paired input
ranknet_pairs = []
ranknet_labels = []

for _, group in grouped:
    if len(group) == 2:  
        row1, row2 = group.iloc[0], group.iloc[1]
        
        # create feature pair
        pair_features_1 = [
            row1['gender_vec'], row1['age_vec'], row1['body_type_vec'], 
            row1['profession_vec'], row1['pet_vec'], row1['pregnant_vec'],
            row1['Intervention_vec'], row1['Barrier_vec'], row1['CrossingSignal_vec']
        ]
        
        pair_features_2 = [
            row2['gender_vec'], row2['age_vec'], row2['body_type_vec'], 
            row2['profession_vec'], row2['pet_vec'], row2['pregnant_vec'],
            row2['Intervention_vec'], row2['Barrier_vec'], row2['CrossingSignal_vec']
        ]
        
        ranknet_pairs.append((pair_features_1, pair_features_2))
        
        # create label,saved==1 > saved==0
        if row1['Saved'] > row2['Saved']:
            ranknet_labels.append(1)  
        else:
            ranknet_labels.append(0)  


In [143]:
from sklearn.model_selection import train_test_split

X = ranknet_pairs  
y = ranknet_labels  

# train_test_validation_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [145]:
import torch
import torch.nn as nn
import torch.optim as optim

class RankNet(nn.Module):
    def __init__(self, input_dim):
        super(RankNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        
    def forward(self, x):
        return self.fc(x)


In [146]:
# build with acutal data
input_dim = 9  # 9 vectorized features
model = RankNet(input_dim)

# loss & optimizer
criterion = nn.BCEWithLogitsLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [149]:

epochs = 10
for epoch in range(epochs):
    model.train()
    
    # train
    for pair, label in zip(X_train, y_train):
        features1, features2 = torch.tensor(pair[0]).float(), torch.tensor(pair[1]).float()  # 转换为 float 类型
        
        # calculate score for each group
        score1 = model(features1)
        score2 = model(features2)
        
        # The goal of RankNet is to make the difference between score1 and score2 reflect the label
        target = torch.tensor([label], dtype=torch.float)  # transform to scalar
        pred = score1 - score2  
        
        loss = criterion(pred, target)  
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for pair, label in zip(X_val, y_val):
            features1, features2 = torch.tensor(pair[0]).float(), torch.tensor(pair[1]).float()  

            score1 = model(features1)
            score2 = model(features2)
            target = torch.tensor([label], dtype=torch.float)  
            pred = score1 - score2
            
            val_loss += criterion(pred, target).item()
    
    print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss/len(X_val)}')

# test
model.eval()
test_loss = 0
with torch.no_grad():
    for pair, label in zip(X_test, y_test):
        features1, features2 = torch.tensor(pair[0]).float(), torch.tensor(pair[1]).float()  
        
        score1 = model(features1)
        score2 = model(features2)
        target = torch.tensor([label], dtype=torch.float) 
        pred = score1 - score2
        
        test_loss += criterion(pred, target).item()

print(f'Test Loss: {test_loss/len(X_test)}')


Epoch 1, Training Loss: 0.957237720489502, Validation Loss: 0.5515197398303426
Epoch 2, Training Loss: 0.8884412050247192, Validation Loss: 0.5508653697524813
Epoch 3, Training Loss: 0.9828553199768066, Validation Loss: 0.550897734190582
Epoch 4, Training Loss: 0.9651614427566528, Validation Loss: 0.5506160302084422
Epoch 5, Training Loss: 0.9227561354637146, Validation Loss: 0.5504825637632111
Epoch 6, Training Loss: 0.9174149632453918, Validation Loss: 0.5509585854145944
Epoch 7, Training Loss: 0.9109825491905212, Validation Loss: 0.55044854541284
Epoch 8, Training Loss: 0.9517418146133423, Validation Loss: 0.5504655188054606
Epoch 9, Training Loss: 0.9082638621330261, Validation Loss: 0.5502392383498977
Epoch 10, Training Loss: 0.9558299779891968, Validation Loss: 0.550381864360465
Test Loss: 0.5502163393344912


In [150]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# validate accuracy, recall, precision, confusion matrix
model.eval()  

predicted_labels = []
true_labels = []

with torch.no_grad():
    for pair, label in zip(X_test, y_test):  
        features1, features2 = torch.tensor(pair[0]).float(), torch.tensor(pair[1]).float()  
        score1 = model(features1)
        score2 = model(features2)
        pred = score1 - score2

        predicted_label = 1 if pred > 0 else 0
        predicted_labels.append(predicted_label)
        true_labels.append(label)

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"Confusion Matrix: \n{conf_matrix}")


Accuracy: 0.7305167905756789
Precision: 0.7153496548337352
Recall: 0.6850732408711021
F1-Score: 0.6998841676982424
Confusion Matrix: 
[[302986  91003]
 [105132 228698]]
